In [1]:
import pandas as pd

df = pd.read_csv('xy_day.csv', 
    dtype={'taxi_id':int, 'location_id':int, 'date_time':object, 'x':float, 'y':float}, 
    parse_dates=['date_time'], 
    infer_datetime_format=True)
df.head(5)

,date_time,taxi_id,location_id,x,y
0,2008-02-02 13:30:00,366,380,17831.228067,10805.626231
1,2008-02-02 13:30:00,719,760,8664.893974,21113.999002
2,2008-02-02 13:30:00,875,876,5281.157698,24207.431410
3,2008-02-02 13:30:00,877,362,1999.227901,10717.910284
4,2008-02-02 13:30:00,950,274,4228.169328,8185.256271


In [2]:
loc = pd.read_csv('xy_loc.csv')
loc.head(5)

,x,y
0,428.127048,416.366323
1,1284.381143,416.366323
2,2140.635234,416.366323
3,2996.889318,416.366323
4,3853.143393,416.366323


In [6]:
def mean_absolute_error(freq, est_freq):
    return (abs(freq - est_freq)).mean()
def mean_relative_error(freq, est_freq):
    freq /= freq.sum()
    est_freq /= est_freq.sum()
    return (abs(freq - est_freq)/(freq + np.finfo(np.float32).eps)).mean()
def jaccard_index(freq, est_freq, k):
    top_k = np.argpartition(-freq, k)[:k]
    est_top_k = np.argpartition(-est_freq, k)[:k]
    return len(np.intersect1d(top_k, est_top_k))/len(np.union1d(top_k, est_top_k))
def MSE(freq, est_freq):
    return (np.square(freq - est_freq)).mean()

In [4]:
import numpy as np
from math import exp

from tqdm import trange

n = 10357
episode = 1440
d = 900
universe = np.arange(d)

w = 30
epsilon = 1

In [7]:
for epsilon in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]:
    threshold = 2000

    window = np.zeros((n, episode), dtype=float)
    # current window pointer
    cwp = 0
    # last-released window pointer
    lwp = np.zeros(n, dtype=int)
    # historical data
    storage = [[] for _ in range(n)]

    current_time = pd.Timestamp('2008-02-02 13:30:00')
    interval = pd.Timedelta('1min')

    util = []
    release_flag_record = []
    for t in trange(episode):
        data = df.loc[df['date_time']==current_time][['location_id', 'taxi_id', 'x', 'y']]
        private_freq = np.zeros(d)
        true_freq = np.zeros(d)
        for row in data.itertuples():
            true_freq[row.location_id] += 1
            # decision
            if storage[row.taxi_id-1]:
                last_release = storage[row.taxi_id-1][len(storage[row.taxi_id-1]) - 1]
                distance = np.sqrt(np.square(row.x - loc.loc[last_release]['x']) + np.square(row.y - loc.loc[last_release]['y']))
                release_flag = (distance > threshold)
                decision_budget = epsilon/(4*w)
                neg_prob = 1/(exp(decision_budget) + 1)
                if np.random.random_sample() < neg_prob:
                    release_flag = ~release_flag
            # if never released
            else:
                release_flag = True
            if release_flag:
                # budget allocation
                forward_projection = w - (cwp - lwp[row.taxi_id-1])
                if storage[row.taxi_id-1] and forward_projection > 0:
                    forward_budget = (epsilon/2 - window[row.taxi_id-1][lwp[row.taxi_id-1]])/forward_projection
                    backward_budget = epsilon/2 - window[row.taxi_id-1][max(0, cwp-w+1):cwp].sum()
                    release_budget = min(forward_budget, backward_budget)
                # if never released or the last release is out of window
                else:
                    release_budget = epsilon/(2*w)
                # randomization
                exp_prob = np.sqrt(np.square(row.x - loc['x'].to_numpy()) + np.square(row.y - loc['y'].to_numpy()))
                exp_prob = np.exp(-release_budget*exp_prob/2)
                exp_prob /= exp_prob.sum()
                private_loc = np.random.choice(universe, p=exp_prob)
                # window update
                window[row.taxi_id-1][cwp] = release_budget
                storage[row.taxi_id-1].append(private_loc)
                release_flag_record.append(row.taxi_id-1)
                lwp[row.taxi_id-1] = cwp
            else:
                approximate_budget = epsilon/(4*w)
                historical_loc = np.array(storage[row.taxi_id-1])
                exp_prob = np.sqrt(np.square(row.x - loc['x'].to_numpy()[historical_loc]) + np.square(row.y - loc['y'].to_numpy()[historical_loc]))
                historical_loc = historical_loc[exp_prob < threshold]
                if historical_loc.size:
                    exp_prob = exp_prob[exp_prob < threshold]
                    exp_prob = np.exp(-approximate_budget*exp_prob/2)
                    exp_prob /= exp_prob.sum()
                    private_loc = np.random.choice(historical_loc, p=exp_prob)
                else:
                    private_loc = last_release
            private_freq[private_loc] += 1
        cwp += 1
        current_time += interval
        # skip empty timestamp
        if np.all(true_freq == 0):
            continue
        # util.append(mean_absolute_error(true_freq, private_freq))
        # util.append(mean_relative_error(true_freq, private_freq))
        # util.append(jaccard_index(true_freq, private_freq, 50))
        util.append(MSE(true_freq, private_freq))
    print(np.mean(util))

100%|██████████| 1440/1440 [05:11<00:00,  4.63it/s]


3.0000078247261346


100%|██████████| 1440/1440 [05:13<00:00,  4.59it/s]


2.430514866979656


100%|██████████| 1440/1440 [05:25<00:00,  4.43it/s]


...

1.2060892018779341


100%|██████████| 1440/1440 [05:10<00:00,  4.64it/s]


1.1507605633802818


100%|██████████| 1440/1440 [05:10<00:00,  4.64it/s]

1.10135524256651
